![](https://www.saa-authors.eu/picture/739/ftw_768/saa-mtcwmza4nzq5mq.jpg)

# Feedback


## How are you after starting refactoring?

## Status of your rewrite?

  * Multiple repos? -> Send a PR on `repositories.py`, we collect all of your repos there
  * You have to implement more than just the API for the simulator! You have to refactor the given *ITU-MiniTwit* application.
  
  
  * Distribute work across all team members.
  * Take small steps, try to work agile.
  
  
  * How do you plan/organize your work? -> You might consider using, e.g., GH issues.
  
  
MSc students:

  * Are you sure that you logged your decisions?

![](http://157.230.24.69/release_activity_weekly.svg)

## Simulator API Specification?

----------------------
# Getting bored today?


![](http://static3.businessinsider.com/image/4fbfb86becad044879000001-506-253/suddenly-startups-have-gotten-very-boring.jpg)




## Recreate the Docker Examples

Recreate the Docker examples, see lecture notes below, to work with any of the follow alternative container engines:

  * [LXC](https://linuxcontainers.org)
  * [rkt](https://coreos.com/rkt/)
  * [FreeBSD Jails](https://www.freebsd.org/doc/handbook/jails.html)
  
<!--
## Write Your Own Container Engine:


  - Choose a programming language and implement a basic container engine.
    - Start by watching Liz Rice's talk where she builds a basic container engine in Go: https://www.youtube.com/watch?v=HPuvDm8IC-4
  - Use the following for inspiration:
    - https://github.com/janoszen/demo-container-runtime
    - https://github.com/p8952/bocker/blob/master/README.md
    - https://www.infoq.com/articles/build-a-container-golang/
    - https://github.com/Zakaria-Ben/Pocker
-->

----------------------


# Mapping Software Dependencies

  > **Software artifact**: A software artifact is a tangible machine-readable document created during software development. Examples are requirement specification documents, design documents, source code and executables.
  >
  > [ISO/IEC 19506:2012 Information technology — Object Management Group Architecture-Driven Modernization (ADM) — Knowledge Discovery Meta-Model (KDM)](https://www.iso.org/obp/ui/#iso:std:iso-iec:19506:ed-1:v1:en)

### Your Turn! -  `Task 1`
<img src="https://media.giphy.com/media/13GIgrGdslD9oQ/giphy.gif" width=50%/>

  - Map the artifacts and all dependencies for the _ITU-MiniTwit_ (the Python 3.8) that you released last week.
  - That is, start with `minitwit.py`, `flag_tool.c`, and `control.sh` and map all of their respective dependencies. 
  - Draw a dependency graph for all artifacts, and discuss with your group fellows if you have covered all artifacts.



  - You might want to use GraphViz (http://www.webgraphviz.com/) to draw your graph quickly and declared as code.
  - Dependency graph? See for example https://en.wikipedia.org/wiki/Dependency_graph

### Helge's take on the task

![](images/minitwit-deps.png)

### Your Turn! -  `Task 2`
<img src="https://media.giphy.com/media/13GIgrGdslD9oQ/giphy.gif" width=50%/>

  - For each edge (arrow) indicate if it is a runtime dependency (*a*) or if it is a build-/compile-time dependency (_b_)
  - Think about if certain dependencies require a specific version of artifact they depend on.

![](images/minitwit-deps2.png)

  - Where do all these artifacts come from?
  - Who knows about all of them?


  - When developing how do you get the required dependencies?
  - In production, i.e., after deployment on a (potentially other) machine how do you get the required dependencies?


  
  - Who is responsible for setting up/configuring all the dependencies?
  - How often do you have to do this?

# Virtualization to tackle development and production dependencies

## Virtual Machines (VM)


  > A hypervisor or virtual machine monitor (VMM) is *computer software*, *firmware* or *hardware* that creates and runs virtual machines. A computer on which a hypervisor runs one or more virtual machines is called a **host machine**, and each virtual machine is called a **guest machine**. 
  >
  > The hypervisor presents the guest operating systems with a virtual operating platform and manages the execution of the guest operating systems. Multiple instances of a variety of operating systems may share the virtualized hardware resources: for example, Linux, Windows, and macOS instances **can all run on a single physical x86 machine**. 
  >
  > This contrasts with **operating-system-level virtualization**, where all instances (usually called containers) must **share a single kernel**, though the guest operating systems can **differ in user space**, such as different Linux distributions with the same kernel.
  >
  > https://en.wikipedia.org/wiki/Hypervisor

![](https://upload.wikimedia.org/wikipedia/commons/e/e1/Hyperviseur.png)




  > Virtual machines run guest operating systems-note the OS layer in each box. This is resource intensive, and the resulting disk image and application state is an entanglement of OS settings, system-installed dependencies, OS security patches, and other easy-to-lose, hard-to-replicate ephemera.
  >
  > (https://docs.docker.com/get-started/#containers-vs-virtual-machines)


<img src="https://docs.docker.com/images/VM%402x.png" width=35%>



There are many different tools for running VMs. Some of them are:

  * [Parallels Workstation](http://www.parallels.com/eu/all-products)
  * [VirtualBox](http://virtualbox.org)
  * [VMware](https://www.vmware.com) 
  * [QEMU](https://www.qemu.org)


We do not have a look at Qemu but you could have used it to run the original _ITU-MiniTwit_ without any modification in a PPC VM, see https://virtuallyfun.com/wordpress/2013/08/15/os-x-powerpc-on-qemu/.

<img src="https://virtuallyfun.com/wordpress/wp-content/uploads/2013/08/Screen-Shot-2013-08-15-at-12.12.08-PM.png" width="60%">

#### Vagrant Multi-machine setup

In the preparation material we created a single VM per `Vagrantfile`. However, you can manage multiple VMs with a single `Vagrantfile`:


```ruby
# -*- mode: ruby -*-
# vi: set ft=ruby :

Vagrant.configure("2") do |config|
  config.vm.box = "generic/ubuntu1804"

  config.vm.network "private_network", type: "dhcp"

  # For two way synchronization you might want to try `type: "virtualbox"`
  config.vm.synced_folder ".", "/vagrant", type: "rsync"

  config.vm.define "dbserver", primary: true do |server|
    server.vm.network "private_network", ip: "192.168.20.2"
    config.vm.network "forwarded_port", guest: 27017, host: 27017
    config.vm.network "forwarded_port", guest: 28017, host: 28017
    server.vm.provider "virtualbox" do |vb|
      vb.memory = "1024"
    end
    server.vm.hostname = "dbserver"
    server.vm.provision "shell", inline: <<-SHELL
        echo "Installing MongoDB"
        wget -qO - https://www.mongodb.org/static/pgp/server-4.2.asc | sudo apt-key add -
        echo "deb [ arch=amd64 ] https://repo.mongodb.org/apt/ubuntu bionic/mongodb-org/4.2 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-4.2.list
        sudo apt-get update
        sudo apt-get install -y mongodb-org

        sudo mkdir -p /data/db
        sudo sed -i '/  bindIp:/ s/127.0.0.1/0.0.0.0/' /etc/mongod.conf

        sudo systemctl start mongod
        mongorestore --gzip /vagrant/dump
    SHELL
  end

  config.vm.define "webserver", primary: true do |server|
    server.vm.network "private_network", ip: "192.168.20.3"
    server.vm.network "forwarded_port", guest: 5000, host: 5000
    server.vm.provider "virtualbox" do |vb|
      vb.memory = "1024"
    end
    server.vm.hostname = "webserver"
    server.vm.provision "shell", inline: <<-SHELL
        export DB_IP="192.168.20.2"

        echo "Installing Anaconda..."
        sudo wget https://repo.anaconda.com/archive/Anaconda3-2019.07-Linux-x86_64.sh -O $HOME/Anaconda3-2019.07-Linux-x86_64.sh
    
        bash ~/Anaconda3-2019.07-Linux-x86_64.sh -b
        
        echo ". $HOME/.bashrc" >> $HOME/.bash_profile
        echo "export PATH=$HOME/anaconda3/bin:$PATH" >> $HOME/.bash_profile
        export PATH="$HOME/anaconda3/bin:$PATH"
        rm Anaconda3-2019.07-Linux-x86_64.sh
        source $HOME/.bash_profile

        pip install Flask-PyMongo

        cp -r /vagrant/* $HOME
        nohup python minitwit.py > out.log &
        echo "================================================================="
        echo "=                            DONE                               ="
        echo "================================================================="
        echo "Navigate in your browser to:"
        echo "http://192.168.20.3:5000"
    SHELL
  end

  config.vm.provision "shell", privileged: false, inline: <<-SHELL
    sudo apt-get update
  SHELL
end
```

Other ways of provisioning, such as with [Ansible Playbooks](https://docs.ansible.com/ansible/latest/user_guide/playbooks.html), [Chef cookbooks](https://docs.chef.io/chef_solo.html), or [Puppet](https://www.puppetlabs.com/puppet).

See https://www.vagrantup.com/docs/provisioning/ for more on different provisioning tools.


#### Different Providers

In this class, I always use VirtualBox as a local provider and DigitalOcean as remote ("cloud") provider. However, you can use many other, such as *VMWare*, *Parallels*, *AWS*, *Azure*, etc. To provision your VMs at remote providers in the cloud, you need usually a corresponding Vagrant plug-in.

```bash
$ vagrant up --provider=digital_ocean
$ vagrant up --provider=aws
$ vagrant up --provider=azure
```

The first line would deploy your VM as droplet on DigitalOcean (https://github.com/devopsgroup-io/vagrant-digitalocean), the second line would do similarly on Amazon AWS (https://github.com/mitchellh/vagrant-aws), and the third line would deploy at Azure (https://github.com/Azure/vagrant-azure).

### Demo: **remote** deployment with Vagrant

See [./README_EXERCISE.md](./README_EXERCISE.md) for descriptions and how to recreate this example in the exercise session.

```bash
$ git clone https://github.com/itu-devops/flask-minitwit-mongodb.git
$ cd flask-minitwit-mongodb
$ git checkout VMify_remote
$ rm -r .vagrant/
$ vagrant up
==> webserver: Running action triggers before up ...
==> webserver: Running trigger...
==> webserver: Waiting to create server until dbserver's IP is available.
==> dbserver: Using existing SSH key: ITU
==> dbserver: Creating a new droplet...
...
```

#### While waiting, let's decipher the `Vagrantfile` for remote deployment

Can you spot an aspect of the remote setup creation that poses a risk on maintainability/reproducability?

```ruby
# -*- mode: ruby -*-
# vi: set ft=ruby :

$ip_file = "db_ip.txt"

Vagrant.configure("2") do |config|
    config.vm.box = 'digital_ocean'
    config.vm.box_url = "https://github.com/devopsgroup-io/vagrant-digitalocean/raw/master/box/digital_ocean.box"
    config.ssh.private_key_path = '~/.ssh/id_rsa'
    config.vm.synced_folder ".", "/vagrant", type: "rsync"
  
    config.vm.define "dbserver", primary: true do |server|
      server.vm.provider :digital_ocean do |provider|
        provider.ssh_key_name = ENV["SSH_KEY_NAME"]
        provider.token = ENV["DIGITAL_OCEAN_TOKEN"]
        provider.image = 'ubuntu-18-04-x64'
        provider.region = 'fra1'
        provider.size = 's-1vcpu-1gb'
        provider.privatenetworking = true
      end
  
      server.vm.hostname = "dbserver"

      server.trigger.after :up do |trigger|
        trigger.info =  "Writing dbserver's IP to file..."
        trigger.ruby do |env,machine|
          remote_ip = machine.instance_variable_get(:@communicator).instance_variable_get(:@connection_ssh_info)[:host]
          File.write($ip_file, remote_ip)
        end 
      end

      server.vm.provision "shell", inline: <<-SHELL
        echo "Installing MongoDB"
        wget -qO - https://www.mongodb.org/static/pgp/server-4.2.asc | sudo apt-key add -
        echo "deb [ arch=amd64 ] https://repo.mongodb.org/apt/ubuntu bionic/mongodb-org/4.2 multiverse" | sudo tee /etc/apt/sources.list.d/mongodb-org-4.2.list
        sudo apt-get update
        sudo apt-get install -y mongodb-org

        sudo mkdir -p /data/db
        sudo sed -i '/  bindIp:/ s/127.0.0.1/0.0.0.0/' /etc/mongod.conf

        sudo systemctl start mongod
        mongorestore --gzip /vagrant/dump
      SHELL
    end

    config.vm.define "webserver", primary: false do |server|
  
      server.vm.provider :digital_ocean do |provider|
        provider.ssh_key_name = ENV["SSH_KEY_NAME"]
        provider.token = ENV["DIGITAL_OCEAN_TOKEN"]
        provider.image = 'ubuntu-18-04-x64'
        provider.region = 'fra1'
        provider.size = 's-1vcpu-1gb'
        provider.privatenetworking = true
      end

      server.vm.hostname = "webserver"

      server.trigger.before :up do |trigger|
        trigger.info =  "Waiting to create server until dbserver's IP is available."
        trigger.ruby do |env,machine|
          while !File.file?($ip_file) do
            sleep(1)
          end
          db_ip = File.read($ip_file).strip()
          puts "Now, I have it..."
          puts db_ip
        end 
      end

      server.trigger.after :provision do |trigger|
        trigger.ruby do |env,machine|
          File.delete($ip_file) if File.exists? $ip_file
        end 
      end

      server.vm.provision "shell", inline: <<-SHELL
        export DB_IP=`cat /vagrant/db_ip.txt`
        echo $DB_IP

        echo "Installing Anaconda..."
        sudo wget https://repo.anaconda.com/archive/Anaconda3-2019.07-Linux-x86_64.sh -O $HOME/Anaconda3-2019.07-Linux-x86_64.sh
    
        bash ~/Anaconda3-2019.07-Linux-x86_64.sh -b
        
        echo ". $HOME/.bashrc" >> $HOME/.bash_profile
        echo "export PATH=$HOME/anaconda3/bin:$PATH" >> $HOME/.bash_profile
        export PATH="$HOME/anaconda3/bin:$PATH"
        rm Anaconda3-2019.07-Linux-x86_64.sh
        source $HOME/.bash_profile

        pip install Flask-PyMongo

        cp -r /vagrant/* $HOME
        nohup python minitwit.py > out.log 2>&1 &
        echo "================================================================="
        echo "=                            DONE                               ="
        echo "================================================================="
        echo "Navigate in your browser to:"
        THIS_IP=`hostname -I | cut -d" " -f1`
        echo "http://${THIS_IP}:5000"
      SHELL
    end
    config.vm.provision "shell", privileged: false, inline: <<-SHELL
      sudo apt-get update
    SHELL
  end
```

----------------

# Hands-on, Containers with Docker

<img src="https://docs.docker.com/images/Container%402x.png" width=35%>



  > A container image is a lightweight, stand-alone, executable package of a piece of software that includes everything needed to run it: code, runtime, system tools, system libraries, settings.
  > 
  > ...
  > 
  > Containers isolate software from its surroundings, for example differences between development and staging environments and help reduce conflicts between teams running different software on the same infrastructure. (https://www.docker.com/what-container)

## Running my first container!

```bash
$ docker run --rm hello-world
```

The command above downloaded the image `hello-world` from the [Docker Hub](https://hub.docker.com), instantiated a container from that image, ran the application within this container, and finally deleted the container (`--rm`).

In [1]:
%%bash
docker run --rm hello-world


Hello from Docker!
This message shows that your installation appears to be working correctly.

To generate this message, Docker took the following steps:
 1. The Docker client contacted the Docker daemon.
 2. The Docker daemon pulled the "hello-world" image from the Docker Hub.
    (amd64)
 3. The Docker daemon created a new container from that image which runs the
    executable that produces the output you are currently reading.
 4. The Docker daemon streamed that output to the Docker client, which sent it
    to your terminal.

To try something more ambitious, you can run an Ubuntu container with:
 $ docker run -it ubuntu bash

Share images, automate workflows, and more with a free Docker ID:
 https://hub.docker.com/

For more examples and ideas, visit:
 https://docs.docker.com/get-started/



Unable to find image 'hello-world:latest' locally
latest: Pulling from library/hello-world
0e03bdcc26d7: Pulling fs layer
0e03bdcc26d7: Verifying Checksum
0e03bdcc26d7: Download complete
0e03bdcc26d7: Pull complete
Digest: sha256:95ddb6c31407e84e91a986b004aee40975cb0bda14b5949f6faac5d2deadb4b9
Status: Downloaded newer image for hello-world:latest


### I need a Linux Shell quickly!

Check on [Docker Hub](https://hub.docker.com), there are images for many different flavors of Linux and for many packaged applications.

```bash
$ docker run -it --rm alpine:latest sh
```

What does that do? It tells Docker to run a container with the latest version of Alpine Linux (a small Linux Distribution), connect to the shell process `sh`, run it interactively `-it` so that you can type in commands and see the results, and finally, to remove the container `--rm` after exiting from the container.





### Volumes

You can mount directories (*volumes*) from your host to a container using the `-v` flag.

```bash
$ docker run -it -v $(pwd):/host alpine:latest /bin/sh
```


### Development with Containers

Let's build a simple webserver in [Go](https://golang.org). To not mess with our development machine we could use a Docker container, which has the Go compiler readily installed.

Find the following `basic_http_server.go` file in the directory `./webserver`.

```go
package main

import (
	"fmt"
	"log"
	"net/http"
)

func helloWorldHandler(w http.ResponseWriter, r *http.Request) {
	fmt.Fprint(w, "Hej verden!\n")
}

func main() {
	port := 8080

	http.HandleFunc("/", helloWorldHandler)

	log.Printf("Server starting on port %v\n", port)
	log.Fatal(http.ListenAndServe(fmt.Sprintf(":%v", port), nil))
}
```

To containerize that program, or better to run that program in a container without installing the compiler to our machines directly, you could run:

```bash
$ docker run -it --rm \
    --name myserver \
    -v $(pwd)/webserver:/src \
    -p 8080:8080 \
    -w /src \
    golang:jessie go run basic_http_server.go
```

That command instantiates a container of the image `golang:jessie` (an Ubuntu Linux with Go and some other necessary tools readily installed). Furthermore, we share our local code in `./webserver` with the container. There it is mounted to the `/src` directory. With `-w` we change the current working directory in that container to `/src`. Additionally, `-p 8080:8080` tells Docker to forward the port `8080` from the container to the same port number on our host. The port number in front of `:` specifies the port on the host, which gets bound to which port of the container (number after `:`). Finally, we run the program within the container `go run basic_http_server.go`.

Note, you could also build the program in the container and run the resulting binary.

```bash
$ docker run -it --rm \
    --name myserver \
    -v $(pwd)/webserver:/src \
    -p 8080:8080 \
    -w /src \
    golang:jessie bash -c "go build basic_http_server.go; ./basic_http_server"
```

Now, you can access the webserver on your host machine on http://127.0.0.1:8080. If you point your browser to that address you should see the following:

![](images/simple_dockerized_webapp.png)

Alternatively, you could run `curl` on your host machine to see that our server is working correctly. 

```bash
$ curl -s http://127.0.0.1:8080
```

Unfortunately, many operating systems do not come with the `curl` program installed. But likely there is a dockerized version of this program. If you run:

```bash
$ docker run --rm \
    --link myserver \
    appropriate/curl:latest curl -s http://myserver:8080
Hej verden!
```

then you downlad an image with a small Linux and with `curl` installed. However, the command above also allows the `curl` client to see our webserver `--link myserver`. Try to run the command above without that flag and see what happens.

To find more Docker images and dockerized programs have a look at https://hub.docker.com.


## `Dockerfile`s

`Dockerfile`s are similar to the `Vagrantfile`s that we discussed earlier. They describe exactly the configuration of a container. Unlike `Vagrantfile`s these configurations are stored as slices on top of each other.

Let's have a look on an example application. It will consist of a webserver and of a simple client. The webserver serves a static HTTP message on port 8080 and the client is just an HTTP `GET` query receiving this message via `curl`. The following UML deployment diagram illustrates this setup

![](images/vm_deployment_basic.png)


Let's have a look at a `Dockerfile` that specifies our webserver.

```Dockerfile
FROM golang:jessie

# Install any needed dependencies...
# RUN go get ...

# Set the working directory
WORKDIR /src

# Copy the server code into the container
COPY basic_http_server.go /src/basic_http_server.go

# Make port 8080 available to the host
EXPOSE 8080

# Build and run the server when the container is started
RUN go build /src/basic_http_server.go
ENTRYPOINT ./basic_http_server
```



As you can see from the above configuration, the `Dockerfile` is similar to everything described in our earlier CLI command:

```bash
$ docker run -it --rm \
    -v $(pwd)/webserver:/src \
    -p 8080:8080 \
    -w /src \
    golang:jessie bash -c "go build basic_http_server.go; ./basic_http_server"
```

Keywords in `Dockerfile`s are `FROM`, `MAINTAINER`, `LABEL`, `RUN`, `CMD`, `EXPOSE`, `ENV`, ADD or `COPY`, `ENTRYPOINT`, `VOLUME`, `USER`, `WORKDIR`, `ONBUILD`. You can read more on them in the documentation: https://docs.docker.com/develop/develop-images/dockerfile_best-practices/.


### Building the Webserver Image

To use containers with our webserver, we first have to build a corresponding image. If you have the above `Dockerfile` stored in a directory `webserver` you can do so as in the following:

```bash
$ cd webserver
$ docker build -t <your_id>/myserver .
```

The `-t` flag tells Docker to build an image with the given name `<your_id>/myserver`. The `.` says: _build the image with the `Dockerfile` in this directory_.

After building your image, you can verify that it is now accessible on your machine.

```bash
$ docker images
REPOSITORY           TAG                 IMAGE ID            CREATED             SIZE
your_id/myserver     latest              a5fe35de13d2        8 seconds ago       704MB
appropriate/curl     latest              f73fee23ac74        3 weeks ago         5.35MB
golang               jessie              6ce094895555        4 weeks ago         699MB
```


### Running the Webserver as Container

```bash
$ docker run --name webserver -p 8080:8080 <your_id>/myserver
```

### Stopping and Restarting the Webserver

```bash
$ docker stop webserver
```

```bash
$ docker start webserver
```


### Building the Client Image


```Dockerfile
FROM appropriate/curl:latest

ENTRYPOINT curl -s http://webserver:8080
```

### Building the Client Image

```bash
$ cd client
$ docker build -t <your_id>/myclient .
```

```bash
$ docker images
REPOSITORY           TAG                 IMAGE ID            CREATED             SIZE
<your_id>/myclient     latest              3714e67fa75a        4 seconds ago       5.35MB
<your_id>/myserver     latest              a5fe35de13d2        About an hour ago   704MB
appropriate/curl     latest              f73fee23ac74        3 weeks ago         5.35MB
golang               jessie              6ce094895555        4 weeks ago         699MB
```




### Running the Webserver as Container

```bash
$ docker run --name client --link webserver <your_id>/myclient
Hej verden!
```

That is nice, is not it? We just build a small application consisting of a webserver and a client, both deployed in their own containers, and we did not have to install any dependencies on our host machine manually.

However, starting the server and the client by hand with the `docker run ...` command is quite tedious. Furthermore, it is not really in line with the _infrastructure as code_ paradigm. Therefore, we can automate even further using `docker-compose`.


## `docker-compose` - Starting the Application Automatically

Similar to a `Vagrantfile`, which describes a cluster setup, we can use a `docker-compose.yml` file to specify the components of our application and how they shall be started.


```yml
version: '3'
services:
  webserver:
    image: your_id/myserver
    ports:
      - "8080:8080"

  clidownload:
    image: appropriate/curl
    links:
      - webserver
    entrypoint: sh -c  "sleep 5 && curl -s http://webserver:8080"
```

```bash
$ docker-compose up
Creating session_03_webserver_1 ... done
Creating session_03_clidownload_1 ... done
Attaching to session_03_webserver_1, session_03_clidownload_1
webserver_1    | 2020/02/12 14:08:03 Server starting on port 8080
clidownload_1  | Hej verden!
session_03_clidownload_1 exited with code 0
^CGracefully stopping... (press Ctrl+C again to force)
Stopping session_03_webserver_1   ... done
```

Finally, to clean up:

```bash
$ docker-compose rm -v
```



# Reflection - Why and for what to use virtualization?


<!--
Solves two problems:
    
  - Setting up a homogeneous development environment
  - Explicit and reproducible way of specifying dependencies and configuration
  
Drawback:

  - These machines are quite big. (But you get a full VM with possibility of doing GUI work directly)
  -
-->

  - a
  - b
  - c
  
<!--  
  - To be HW independent (VM)
  - Scalability
  - Minimizing env. config.
  - Reproducability
  - Upgradability
-->

## Advantages and drawbacks of VMs?


## Advantages and drawbacks of containers?

# What to do now?

  * To prepare for your project work, practice with the [exercises](../session_03/README_EXERCISE.md)
  * Do the [project work](../session_03/README_TASKS.md) until the end of the week
  * And [prepare for the next session](../session_04/README_PREP.md)